In [65]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from model import MyXgbLgbCat
from sklearn.model_selection import KFold
import numpy as np
from sklearn.preprocessing import OrdinalEncoder
%run model.py

In [66]:
X = datasets.fetch_covtype().data[:3000]
y = datasets.fetch_covtype().target[:3000]  # 多分类任务
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [67]:
enc = OrdinalEncoder()
y_train_new = enc.fit_transform(y_train.reshape(-1, 1))
np.unique(y_train_new)

array([0., 1., 2., 3., 4., 5., 6.])

In [68]:
y_test_new = enc.transform(y_test.reshape(-1, 1))
y_test_new

array([[0.],
       [1.],
       [4.],
       [0.],
       [0.],
       [6.],
       [1.],
       [4.],
       [4.],
       [6.],
       [1.],
       [5.],
       [2.],
       [2.],
       [0.],
       [2.],
       [4.],
       [2.],
       [4.],
       [4.],
       [4.],
       [1.],
       [5.],
       [1.],
       [2.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [4.],
       [1.],
       [0.],
       [5.],
       [5.],
       [1.],
       [0.],
       [4.],
       [5.],
       [1.],
       [4.],
       [5.],
       [0.],
       [1.],
       [1.],
       [5.],
       [0.],
       [2.],
       [5.],
       [1.],
       [0.],
       [4.],
       [4.],
       [6.],
       [5.],
       [3.],
       [1.],
       [0.],
       [2.],
       [0.],
       [2.],
       [4.],
       [0.],
       [1.],
       [4.],
       [4.],
       [0.],
       [4.],
       [0.],
       [1.],
       [4.],
       [1.],
       [6.],
       [2.],
       [4.],
       [0.],

In [69]:
kfold = KFold(n_splits=5, random_state=66, shuffle=True)

xgb = XGBClassifier
xgb_params = {'n_jobs': -1, "use_label_encoder": False}
xgb_fit_params = {'early_stopping_rounds': 100, "verbose": False,
                  "eval_metric": 'mlogloss'}

lgb = LGBMClassifier
lgb_params = {'n_jobs': -1}
lgb_fit_params = {"early_stopping_rounds": 100, "verbose": False,
                  "eval_metric": 'binary_logloss'}

cat = CatBoostClassifier
cat_params = {'thread_count': -1,
               "allow_writing_files": False,
               "verbose": False}
cat_fit_params = {"early_stopping_rounds": 100}

In [70]:
models_lgb, score_lst_lgb = MyXgbLgbCat(X_train, y_train_new.reshape(-1, ), model=lgb,
                                        kf=kfold, model_params=lgb_params,
                                        fit_params=lgb_fit_params)

KFold: 0
LGBMClassifier_score:0.8444444444444444
KFold: 1
LGBMClassifier_score:0.8644444444444445
KFold: 2
LGBMClassifier_score:0.8311111111111111
KFold: 3
LGBMClassifier_score:0.8222222222222222
KFold: 4
LGBMClassifier_score:0.8377777777777777


In [71]:
models_cat, score_lst_cat = MyXgbLgbCat(X_train, y_train_new, model=cat,
                                        kf=kfold, model_params=cat_params,
                                        fit_params=cat_fit_params)

KFold: 0
CatBoostClassifier_score:0.19984197530864198
KFold: 1
CatBoostClassifier_score:0.21154074074074075
KFold: 2
CatBoostClassifier_score:0.2084888888888889
KFold: 3
CatBoostClassifier_score:0.20535802469135803
KFold: 4
CatBoostClassifier_score:0.2142716049382716


In [72]:
models_xgb, score_lst_xgb = MyXgbLgbCat(X_train, y_train_new, model=xgb,
                                        kf=kfold, model_params=xgb_params,
                                        fit_params=xgb_fit_params)

KFold: 0
**************
XGBClassifier_score:0.8333333333333334
KFold: 1
**************
XGBClassifier_score:0.8555555555555555
KFold: 2
**************
XGBClassifier_score:0.8311111111111111
KFold: 3
**************
XGBClassifier_score:0.8155555555555556
KFold: 4
**************
XGBClassifier_score:0.8466666666666667
